In [13]:
from OSMPythonTools.nominatim import Nominatim
from pathlib import Path
import pandas as pd
import re

df = pd.read_csv("../datasets/Faramacie_temp.csv", sep=";")
nominatim = Nominatim()
results = []

LAT = []
LON = []

for via, citta in zip(df["INDIRIZZO"], df["DESCRIZIONECOMUNE"]):

    via = str(via).lower().replace('a\'', 'a').replace('e\'', 'e').replace('i\'', 'i').replace('o\'', 'o').replace('u\'', 'u')
    citta = str(citta).lower().replace('a\'', 'a').replace('e\'', 'e').replace('i\'', 'i').replace('o\'', 'o').replace('u\'', 'u')

    res = nominatim.query(via + "," + citta)
    results.append(res)

for item in results:
    tmp = item.toJSON()
    if len(tmp) != 0:
        LAT.append(tmp[0]['lat'])
        LON.append(tmp[0]['lon'])
    else:
        LAT.append(0.0)
        LON.append(0.0)

df["LONGITUDINE"] = pd.Series(LON)
df["LATITUDINE"] = pd.Series(LAT)
#df.insert(loc=0, column='LONGITUDINE', value=pd.Series(LON))
#df.insert(loc=0, column='LATITUDINE', value=pd.Series(LAT))

filepath = Path('../datasets/out.csv')
df.to_csv(filepath, index=False)

[nominatim] downloading data: search


In [14]:
import pandas as pd
from pathlib import Path
farmacie = pd.read_csv("../datasets/pivoted/farmacie.csv", sep=";")
farmacie["LATITUDINE"] = farmacie["LATITUDINE"].apply(lambda x : float(x.replace(',', '.')) if x != '-' else 0.0)
farmacie["LONGITUDINE"] = farmacie["LONGITUDINE"].apply(lambda x : float(x.replace(',', '.')) if x != '-' else 0.0)
farmacie["PARTITAIVA"] = farmacie["PARTITAIVA"].apply(lambda x : x if x != "-" else 0)
farmacie["INDIRIZZO"] = farmacie["INDIRIZZO"].apply(lambda x : x.replace('"', ''))
filepath = Path('../datasets/completed/farmacie.csv')
farmacie.to_csv(filepath, index=False,sep=";")

In [15]:
import numpy as np
import pandas as pd
farmacie = pd.read_csv("../datasets/completed/farmacie.csv", sep=";")
farmacie.convert_dtypes()
farmacie.dtypes

SIGLAPROVINCIA           object
DESCRIZIONEPROVINCIA     object
DESCRIZIONECOMUNE        object
CAP                       int64
INDIRIZZO                object
DESCRIZIONEFARMACIA      object
LATITUDINE              float64
LONGITUDINE             float64
PARTITAIVA                int64
dtype: object

In [19]:
df = pd.read_csv("../datasets/out.csv")
df.dtypes
print(df.head())

  SIGLAPROVINCIA DESCRIZIONEPROVINCIA        DESCRIZIONECOMUNE    CAP  \
0             RG               RAGUSA                   RAGUSA  97010   
1             TP              TRAPANI  CASTELLAMMARE DEL GOLFO  91014   
2             RG               RAGUSA                 VITTORIA  97019   
3             SR             SIRACUSA                 FLORIDIA  94014   
4             CT              CATANIA                  SCORDIA  95048   

                           INDIRIZZO  \
0                   Via Pozzallo, 99   
1                 Corso Garibaldi 18   
2             Stradale Scoglitti, 66   
3           Viale Vittorio Veneto 15   
4  Via Tenente De Cristofaro N.37/39   

                                 DESCRIZIONEFARMACIA  LATITUDINE  LONGITUDINE  \
0  FARMACIE DOTT. SCHEMBARI GIORGIO E LUCIO RIUNI...   36.785224    14.555302   
1                         Farmacia Del Centro S.r.l.   38.020555    12.881009   
2                                  Farmacia Vittoria    0.000000     0.000000

In [20]:
import pandas as pd
from pathlib import Path
farmacie = pd.read_csv("../datasets/completed/farmacie.csv", sep=";")
farmacie = farmacie.drop(farmacie.loc[1484:].index)

print(farmacie.head())
df = pd.read_csv("../datasets/out.csv")
df_finale = pd.merge(farmacie,df,how="outer")


  SIGLAPROVINCIA DESCRIZIONEPROVINCIA DESCRIZIONECOMUNE    CAP  \
0             PA              PALERMO           PALERMO  90135   
1             AG            AGRIGENTO         AGRIGENTO  92100   
2             AG            AGRIGENTO         AGRIGENTO  92100   
3             AG            AGRIGENTO         AGRIGENTO  92100   
4             AG            AGRIGENTO         AGRIGENTO  92100   

                    INDIRIZZO  \
0  Via EUGENIO L'EMIRO, 34/36   
1           Via Empedocle, 71   
2   Via Leonardo Sciascia, 28   
3  Via Alessandro Manzoni, 28   
4             Via Gioeni, 159   

                                 DESCRIZIONEFARMACIA  LATITUDINE  LONGITUDINE  \
0                                          VERGA SRL   45.758034     8.376776   
1                                     AVERNA ANTONIO   37.310958    13.581285   
2  CAMILLERI DELLE DR.SSE ANNACAMILLERI E DANIELA...   37.273686    13.617993   
3                                     NUARA MARCELLA   37.308149    13.580227   

In [21]:
filepath = Path('../datasets/farmacie_new.csv')
df_finale.to_csv(filepath, index=False)